In [10]:
!pip install pandas scikit-learn matplotlib seaborn


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\maghr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
def read_reviews(data_dir):
    reviews = []
    sentiments = []

    for sentiment in ['pos', 'neg']:
        sentiment_dir = os.path.join(data_dir, sentiment)
        for filename in os.listdir(sentiment_dir):
            with open(os.path.join(sentiment_dir, filename), 'r', encoding='utf-8') as f:
                reviews.append(f.read())
                sentiments.append(sentiment)
    return reviews, sentiments

train_data_dir = 'data/aclImdb/train'
train_reviews, train_sentiments = read_reviews(train_data_dir)

test_data_dir = 'data/aclImdb/test'
test_reviews, test_sentiments = read_reviews(test_data_dir)

train_df = pd.DataFrame({'review': train_reviews, 'sentiment':train_sentiments})
test_df = pd.DataFrame({'review': test_reviews, 'sentiment': test_sentiments})

print(train_df.head())
print(test_df.head())

                                              review sentiment
0  Bromwell High is a cartoon comedy. It ran at t...       pos
1  Homelessness (or Houselessness as George Carli...       pos
2  Brilliant over-acting by Lesley Ann Warren. Be...       pos
3  This is easily the most underrated film inn th...       pos
4  This is not the typical Mel Brooks film. It wa...       pos
                                              review sentiment
0  I went and saw this movie last night after bei...       pos
1  Actor turned director Bill Paxton follows up h...       pos
2  As a recreational golfer with some knowledge o...       pos
3  I saw this film in a sneak preview, and it is ...       pos
4  Bill Paxton has taken the true story of the 19...       pos
